In [599]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh
%matplotlib inline
from __future__ import division

In [600]:
def Overlap_ij(alpha,i,j):
    Sij = ( np.pi / (alpha[i] + alpha[j]) )**(1.5)
    return Sij

def Matriz_Overlap(alpha):
    nsize = len(alpha)
    S = np.zeros((nsize,nsize))
    for i in range(0,nsize):
        for j in range(i,nsize):        
            S[i,j]=Overlap_ij(alpha,i,j)
            S[j,i]=S[i,j]
    return S

In [1000]:
# Elemento de matriz de cinetica
def Tij(alpha,l,i,j):
    rnum = 3 * np.pi**(1.5) * alpha[i] * alpha[j]
    rden = (alpha[i] + alpha[j])**(2.5)    
    Tij = rnum/rden + l*(l+1)*np.pi**(3/2)/(alpha[i]+alpha[j])**0.5
    return Tij


# Elemento de matriz de potencial
def Vij(alpha,Z,i,j):
    Vij =  Z*( -2*np.pi )  /  (alpha[i] + alpha[j])    
    return Vij

# Matriz del Hamiltoniano en la base
def Matriz_H(alpha,l,Z):
    nsize = len(alpha)
    H = np.zeros((nsize,nsize))
    for i in range(nsize):
        for j in range(i,nsize): 
            H[i,j] = Tij(alpha,l,i,j) + Vij(alpha,Z,i,j)
            H[j,i] = H[i,j]
    return H

def Qprqs(alpha,p,r,q,s):
    num = 2*np.pi**2.5 
    den = (alpha[p]+alpha[q])*(alpha[r]+alpha[s])*(alpha[p]+alpha[q]+alpha[r]+alpha[s])**0.5
    return num/den

def Tensor_Q(alpha):
    Iter = range(len(alpha))
    Q = np.array([[[[Qprqs(alpha,p,r,q,s) for p in Iter] for r in Iter] for q in Iter] for s in Iter])
    return Q

def F_Matriz(H,Q,Ci):
    size = H.shape[0]
    F = np.zeros(H.shape)
    for i in range(size):
        for j in range(i,size):
            F[i,j] = H[i,j] + Ci.dot(Q[i,:,j,:].dot(Ci))
            F[j,i] = F[i,j]
    return F

def C_Norm(C,S):
    size = C.shape[0]
    for i in range(size):
        Ci = C[:,i]
        norm = Ci.T.dot(S.dot(Ci))**0.5
        C[:,i] = C[:,i]/norm

def Energy_Calc(Ci,H,Q):
    Iter = range(Ci.shape[0])
    cumres = 2*Ci.T.dot(H.dot(Ci))
    for p in Iter:
        for r in Iter:
            for q in Iter:
                for s in Iter:
                    cumres = cumres + Ci[p]*Ci[r]*Ci[q]*Ci[s]*Q[p,r,q,s]
    return cumres

In [1001]:
alpha = np.array([0.298073,1.242567,5.782948,38.474970])
Z = 2
l = 1

C_new = np.ones((4,4)) # np.random.rand(4,4)
C_Norm(C_new,S)

H = Matriz_H(alpha,l,Z)
Q = Tensor_Q(alpha)
S = Matriz_Overlap(alpha)
cont = 0

In [1007]:
C = C_new
F = F_Matriz(H,Q,C[:,0])
E, C_new = eigh(F,S,type=1)
C_Norm(C_new,S)
print "Iteracion Nro:", cont
print "-) Energia Total:", Energy_Calc(C_new[:,0],H,Q)
print "-) Energia Ionizacion:", E[0]
cont += 1

Iteracion Nro: 5
-) Energia Total: -0.18314058913957496
-) Energia Ionizacion: 0.17817109144611457
